In [13]:
# Instalando herramientas necesarias
!pip install nltk


In [14]:
# Importamos las librerías a usar
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [15]:
# Importamos el CSV original

df_original = pd.read_csv('/content/claves_cfdi.csv')

In [16]:
# Verificamos el contenido

df_original.head(5)

,c_ClaveProdServ,Descripción,Incluir IVA trasladado,Incluir IEPS trasladado,Complemento que debe incluir,FechaInicioVigencia,FechaFinVigencia,Estímulo Franja Fronteriza,Palabras similares
0,1010101,No existe en el catálogo,Opcional,Opcional,NaN,1/1/2022,NaN,0,Público en general
1,10101500,Animales vivos de granja,Opcional,Opcional,NaN,1/1/2022,NaN,1,NaN
2,10101501,Gatos vivos,Opcional,Opcional,NaN,1/1/2022,NaN,1,NaN
3,10101502,Perros,Opcional,Opcional,NaN,1/1/2022,NaN,1,NaN
4,10101504,Visón,Opcional,Opcional,NaN,1/1/2022,NaN,1,NaN


In [17]:
# Extraemos solo las columnas que necesitamos

df = df_original[['c_ClaveProdServ','Descripción','Palabras similares']]

df.head(5)

,c_ClaveProdServ,Descripción,Palabras similares
0,1010101,No existe en el catálogo,Público en general
1,10101500,Animales vivos de granja,NaN
2,10101501,Gatos vivos,NaN
3,10101502,Perros,NaN
4,10101504,Visón,NaN


In [18]:
# Renombramos las columnas para mayor facilidad
df.columns = ['clave', 'descripcion', 'alternativas']

df.head(5)


,clave,descripcion,alternativas
0,1010101,No existe en el catálogo,Público en general
1,10101500,Animales vivos de granja,NaN
2,10101501,Gatos vivos,NaN
3,10101502,Perros,NaN
4,10101504,Visón,NaN


Eliminaremos los stopwords de las descripciones normales y alternativas pues no consideramos que nos ayuden
en la clasificación posterior del proyecto.

Para esto usaremos NLTK.


In [19]:
nltk.download('stopwords')
nltk.download('punkt')

stopwords_es = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
# Creamos una función para realizar el preprocesamiento básico de nuestras descripciones

def preprocesamiento_texto(texto):
  if texto:
    texto_sin_puntuacion = re.sub(r'[^\w\s]', '', texto)
    palabras = word_tokenize(texto_sin_puntuacion)
    palabras_filtradas = [palabra for palabra in palabras if palabra.lower() not in stopwords_es]
    return str(" ".join(palabras_filtradas))
  else:
    return ""

In [21]:
# Aplicamos nuestro preprocesamiento a las columnas

df.fillna("",inplace=True)
df.loc[:, 'descripcion_ok'] = df['descripcion'].apply(preprocesamiento_texto)
df.loc[:, 'alternativas_ok'] = df['alternativas'].apply(preprocesamiento_texto)

<ipython-input-21-9754add1a32d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna("",inplace=True)
<ipython-input-21-9754add1a32d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'descripcion_ok'] = df['descripcion'].apply(preprocesamiento_texto)
<ipython-input-21-9754add1a32d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [22]:
# Verificamos que se haya aplicado el preprocesamiento
df.head(10)

,clave,descripcion,alternativas,descripcion_ok,alternativas_ok
0,1010101,No existe en el catálogo,Público en general,existe catálogo,Público general
1,10101500,Animales vivos de granja,,Animales vivos granja,
2,10101501,Gatos vivos,,Gatos vivos,
3,10101502,Perros,,Perros,
4,10101504,Visón,,Visón,
5,10101505,Ratas,,Ratas,
6,10101506,Caballos,"Equinos, Potrancas, Potras, Potrillos, Potros,...",Caballos,Equinos Potrancas Potras Potrillos Potros Yeguas
7,10101507,Ovejas,"Borregos, Carneros",Ovejas,Borregos Carneros
8,10101508,Cabras,"Borregos cimarrones, Cabritos, Cabros, Carnero...",Cabras,Borregos cimarrones Cabritos Cabros Carnero Ro...
9,10101509,Asnos,"Borricos, Burros",Asnos,Borricos Burros


In [23]:
# Procederemos a probar generar los embeddings con BERT

from transformers import AutoTokenizer, AutoModel
import torch

# Cargamos el modelo preentrenado y el tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def generar_embeddings(texto):
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embeddings


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
df['desc_embeddings'] = df['descripcion'].apply(generar_embeddings)


In [ ]:
df.to_csv('dataframe_con_embeddings.csv', index=False)


# Conclusiones

Es bien entendido que en la metodología CRISP-ML (Cross Industry Standard Process for Machine Learning), la fase de preparación de datos es fundamental y crítica para asegurar que los datos estén listos para ser utilizados y las principales conclusiones que podemos aportar son las siguientes:

Datos relevantes seleccionados: Hacemos una selección de las características más importantes. Esto incluye reducir la dimensionalidad eliminando las variables irrelevantes o redundantes, lo que mejora el rendimiento del modelo y reduce el riesgo de sobreajuste.

Calidad de los datos mejorada: Después de limpiar y transformar los datos, logramos una base de datos más homogénea y libre de ruido, lo que nos ayuda  que los modelos trabajen con información precisa. Esto nos permite eliminar valores nulos, duplicados y corregir inconsistencias.

Formateo adecuado para los algoritmos: Dentro de la preparación incluye transformar los datos para que se ajusten al formato que requieren los algoritmos. Esto puede incluir codificación de variables categóricas, normalización entre otras

Reducción del sesgo y la varianza: Con la  preparación de datos, buscamos  minimizar problemas que podrían introducir sesgos en los modelos, como la falta de representación adecuada de ciertos grupos de datos, o problemas de varianza alta debido a la mala distribución de las variables de nuestro modelo.

Datos consistentes para la fase de modelado: es importante considerar que al final de la fase de preparación, los datos deben estar en un formato que permita una transición suave a la fase de modelado, así podemos asegurar que no se requieran modificaciones adicionales que puedan complicar el proceso de desarrollo del mismo

Creación de nuevas características:  es importante considerar que esta fase también puede incluir la creación de nuevas variables a partir de las ya existentes, mejorando el poder predictivo de los modelos.
Para resumir, esta fase asegura que los datos sean fiables, relevantes y adecuados para los algoritmos , mejorando tanto la eficiencia como la precisión de los modelos.
